<a href="https://colab.research.google.com/github/F1ameX/TUSUR_ML/blob/main/TUSUR_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Первый вариант:

**Задача**: Использовать предобученную языковую модель для генерации продолжения текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую GPT-модель для генерации продолжения текста с заданными параметрами. Если модель сгенерировала неполное предложение (без точки/восклицательного знака/знака вопроса/многоточия), то удалить эту часть и оставить только полные предложения. Затем убрать параметр repetition_penalty и сравнить качество генерации.

1. Загрузить библиотеку transformers
2. Загрузить модель и токенизатор для модели [ai-forever/rugpt3small_based_on_gpt2](https://huggingface.co/ai-forever/rugpt3small_based_on_gpt2)
3. Написать 5 незаконченных предложений.
4. При вызове метода generate() указать [максимальную длину сгенерированных токенов](https://huggingface.co/docs/transformers/en/main_classes/text_generation) = 150 и repetition_penalty=1.2
5. Передать в модель текст для продолжения.
6. Получить сгенерированный текст и обрезать его до полных предложений (./!/?/...)
7. Убрать параметр repetition_penalty=1.2 и получить второй набор сгенерированных предложений.
8. Сравнить полученные продолжения и сформулировать разницу в качестве полученных генераций. Вывод должен быть сформулирован в отдельной ячейке.

In [2]:
!pip install --quiet transformers

In [3]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [10]:
unfinished_sentences = ['Это будет самое неожиданное первое',
                        'Я не думаю, что завтра не ',
                        'моего преподавателя по физике зовут сэр',
                        'контрольная по дифференциальным уравнениям началась с того, что',
                        'чювачелло, ты видел как']

tokenized_sentences = tokenizer(unfinished_sentences, padding = True, truncation = True, return_tensors = 'pt')

In [12]:
full_sentences = model.generate(
    input_ids = tokenized_sentences['input_ids'],
    max_length = 150,
    repetition_penalty = 1.2
)

In [39]:
import re

for idx in range(len(unfinished_sentences)):
  result = tokenizer.decode(full_sentences[idx], skip_special_tokens = True)
  result = re.split(r'[.!?]', result)[0].replace('\n', '')
  print(result)

Это будет самое неожиданное первое впечатление
Я не думаю, что завтра не встретимся
моего преподавателя по физике зовут сэр Джон
контрольная по дифференциальным уравнениям началась с того, что в ней были введены два новых понятия: "сверхнормативная" и "нормальная"
чювачелло, ты видел как он в последний раз смотрел на меня


In [42]:
full_sentences = model.generate(
    input_ids = tokenized_sentences['input_ids'],
    max_length = 150,
)

In [43]:
import re

for idx in range(len(unfinished_sentences)):
  result = tokenizer.decode(full_sentences[idx], skip_special_tokens = True)
  result = re.split(r'[.!?]', result)[0].replace('\n', '')
  print(result)

Это будет самое неожиданное первое впечатление
Я не думаю, что завтра не встретимся
моего преподавателя по физике зовут сэр Джон
контрольная по дифференциальным уравнениям началась с того, что в ней были введены уравнения, которые были введены в действие в начале XX века
чювачелло, ты видел как он в последний раз смотрел на меня


# Второй вариант

**Задача**: Использовать предобученную языковую модель для оценки тональности текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую BERT-модель для определения степени токсичности высказывания.

1. Загрузить библиотеку transformers
2. Загрузить модель и токенизатор для модели [s-nlp/russian_toxicity_classifier](https://huggingface.co/s-nlp/russian_toxicity_classifier)
3. Написать в сумме 10 предложений, где присутствуют положительные, нейтральные и негативные высказывания.
4. Получить предсказания и вывести их в формате "Текст: класс: оценка"
5. Описать шаги для получения предсказаний.
6. Проанализировать результаты и указать, имеет ли модель склонность оценивать нейтральные комментарии ближе к отрицательным высказываниям. Вывод должен быть сформулирован в отдельной ячейке.